## Tarea 2 - Aprendizaje Automático
Vicente Mieres

In [65]:
# importacion de librerias

import tensorflow as tf
import numpy as np
# from tensorflow.keras.datasets import mnist
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score


# Parte 1 - Lectura de Imágenes

In [2]:
def load_image_paths_and_labels(file_path):
    """Carga las rutas de las imágenes y las etiquetas desde un archivo de texto."""
    image_paths = []
    labels = []
    with open(file_path, 'r') as file:
        for line in file.readlines():
            path, label = line.strip().split('\t')  
            image_paths.append(path)
            labels.append(int(label)) 
    return image_paths, np.array(labels)

def load_images(image_paths, folder_route):
    """Carga las imágenes y las aplana a vectores."""
    images = []
    for path in image_paths:
        with Image.open(folder_route + path) as img:
            img_array = np.array(img).reshape(-1)
            images.append(img_array)
    return np.array(images)

QuickDraw-10

In [39]:
# Cargar las rutas de entrenamiento y prueba junto con las etiquetas
train_image_paths_10, train_labels_10 = load_image_paths_and_labels("./QuickDraw-10/train.txt")
test_image_paths_10, test_labels_10 = load_image_paths_and_labels("./QuickDraw-10/test.txt")

# Cargar y procesar las imágenes
train_images_10 = load_images(train_image_paths_10, "./QuickDraw-10/")
test_images_10 = load_images(test_image_paths_10, "./QuickDraw-10/")

In [40]:
len(train_images_10) / 10

983.4

In [9]:
len(test_images_10)

1166

In [41]:
# Preparacion datos de validacion
train_images_10, val_image_10, train_labels_10, val_labels_10 = train_test_split(
    train_images_10, train_labels_10, test_size=0.15, random_state=42, stratify=train_labels_10
)

In [43]:
len(train_images_10)

8358

In [44]:
len(val_image_10)

1476

In [45]:
len(val_labels_10)

1476

QuickDraw-Animals

In [4]:
def load_images_and_labels(dataset):
    """
    Carga imágenes y etiquetas para train y test del dataset QuickDraw-Animals.
    
    Retorna: 
      X_test, y_test, X_train, y_train (numpy arrays)
    """
    if dataset != "Animals":
        raise ValueError("Sólo soporta dataset 'Animals'.")

    base_path = './QuickDraw-Animals'
    mapping_file = os.path.join(base_path, 'mapping.txt')

    # Leer mapping.txt y crear diccionario etiqueta->número
    label_map = {}
    with open(mapping_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 2:
                label = parts[0]
                idx = int(parts[1])
                label_map[label] = idx

    def load_images_from_folder(folder_path):
        images = []
        labels = []
        # Las subcarpetas son las clases
        for label_name in sorted(os.listdir(folder_path)):
            label_folder = os.path.join(folder_path, label_name)
            if os.path.isdir(label_folder) and label_name in label_map:
                for img_file in sorted(os.listdir(label_folder)):
                    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        img_path = os.path.join(label_folder, img_file)
                        with Image.open(img_path) as img:
                            img_array = np.array(img).reshape(-1)
                            images.append(img_array)
                            labels.append(label_map[label_name])
        return np.array(images), np.array(labels)

    # Cargar test
    X_test, y_test = load_images_from_folder('./QuickDraw-Animals/test_images/test_images')

    # Cargar train
    X_train, y_train = load_images_from_folder('./QuickDraw-Animals/train_images/train_images')

    return X_test, y_test, X_train, y_train


In [51]:
test_images_animals, test_labels_animals, train_images_animals, train_labels_animals = load_images_and_labels("Animals")

In [52]:
len(train_images_animals) / 10

1200.0

In [53]:
len(test_images_animals)

2399

In [54]:
# Preparacion datos de validacion
train_images_animals, val_images_animals, train_labels_animals, val_labels_animals = train_test_split(
    train_images_animals, train_labels_animals, test_size=0.15, random_state=42, stratify=train_labels_animals
)

In [55]:
len(train_images_animals)

10200

In [56]:
len(val_images_animals)

1800

## Parte 2 - Construccion de modelos

Clase MLP

In [63]:
class MLP(tf.keras.Model):
    # defining components
    def __init__(self, layers_size, n_classes, activation='sigmoid'):
        super(MLP, self).__init__()
        self.layer_list = []
        for lsize in layers_size:
            self.layer_list.append(tf.keras.layers.Dense(lsize))
        self.classifier = tf.keras.layers.Dense(n_classes)
        self.activation = activation


    # defining architecture
    def call(self, inputs):
        x = inputs
        for mlp_layer in self.layer_list:
            x = mlp_layer(x)
            if self.activation == 'sigmoid':
                x = tf.keras.activations.sigmoid(x)
            elif self.activation == 'relu':
                x = tf.keras.activations.relu(x)
            elif self.activation == 'tanh':
                x = tf.keras.activations.tanh(x)
            else:
                raise ValueError("Activación no soportada")
        x = self.classifier(x)
        return tf.keras.activations.softmax(x)

Funciones para entrenar, evaluar y realizar los experimentos

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, loss_fn, epochs=10, batch_size=32):
    # Entrenamiento
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=loss_fn,
        metrics=['accuracy']
    )
    final_model = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0
    )
    return final_model

def evaluate_model(model, X_test, y_test, class_names):
    # Predicciones
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)

    acc_total = accuracy_score(y_test, y_pred)

    # Accuracy por clase
    acc_per_class = {}
    for cls in np.unique(y_test):
        idx = y_test == cls
        acc = accuracy_score(y_test[idx], y_pred[idx])
        acc_per_class[class_names[cls]] = acc
  
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)

    return acc_total, acc_per_class, cm

def experiment(X_train, y_train, X_val, y_val, X_test, y_test, layers_size, n_classes, activation, loss_fn, class_names, epochs=10, batch_size=None):
    acc_totals = []
    acc_classes_list = []
    cm_list = []

    for i in range(5):
        print(f"\nEntrenamiento número {i+1}")
        # Crear modelo nuevo para reinicializar pesos
        model = MLP(layers_size, n_classes, activation)
        
        # Entrenar
        train_model(model, X_train, y_train, X_val, y_val, loss_fn, epochs, batch_size)
        
        # Evaluar
        acc_total, acc_per_class, cm = evaluate_model(model, X_test, y_test, class_names)
        acc_totals.append(acc_total)
        acc_classes_list.append(acc_per_class)
        cm_list.append(cm)
    
    # Calcular mediana del accuracy total
    median_acc_total = np.median(acc_totals)
    print(f"\nMediana de accuracy total tras 5 entrenamientos: {median_acc_total:.4f}")
    
    return median_acc_total, acc_classes_list, cm_list

In [85]:
# Experimento para quickdraw-10.1
clases = ['bandage', 'blackberry', 'castle', 'flashlight', 'lion', 'remote-control', 'sink', 'spreadsheet', 'teapot', 'trombone']

acc_total, acc_clase, cm = experiment(
        train_images_10, 
        train_labels_10, 
        val_image_10, 
        val_labels_10, 
        test_images_10, 
        test_labels_10, 
        [512, 256], 
        n_classes=10, 
        activation='relu', 
        loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        class_names=clases,
        epochs=20,
        batch_size=1000
        )


Entrenamiento número 1
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step

Entrenamiento número 2
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step

Entrenamiento número 3
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step

Entrenamiento número 4
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step

Entrenamiento número 5
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step

Mediana de accuracy total tras 5 entrenamientos: 0.2864


In [89]:
acc_total, acc_clase, cm = experiment(
        train_images_10, 
        train_labels_10, 
        val_image_10, 
        val_labels_10, 
        test_images_10, 
        test_labels_10, 
        [512, 256, 128], 
        n_classes=10, 
        activation='sigmoid', 
        loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        class_names=clases,
        epochs=150,
        batch_size=100
        )


Entrenamiento número 1


KeyboardInterrupt: 